<a href="https://colab.research.google.com/github/Yesn-t/TI_1_Practica4/blob/main/TI_1_Practica4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 4


**Nombre:** Amaro Lechuga Jashua Ricardo
**e-mail:** jashua.amaro3877@alumnos.udg.mx

# Modules

In [10]:
import panel as pn
import panel.widgets as pnw

pn.extension('plotly')

import pandas as pd
import numpy as np

import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

import math
from scipy.spatial import distance

# Clases

In [12]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

# Functions

## Trajectory

In [13]:
def Trajectory_DF(Trajectory_Selection, n_steps, x_pos, y_pos, speed, cauchy_coef, levy_exp):
  df = {}
  s_pos = [int(x_pos), int(y_pos)]

  if (Trajectory_Selection == 'BM') :
    df = BM_2d(n_steps, speed, s_pos)

  elif (Trajectory_Selection == 'CRW'):
    df = CRW_2d(n_steps, speed, s_pos, cauchy_coef)

  elif (Trajectory_Selection == 'LF'):
    df = LF_2d(n_steps, speed, cauchy_coef, levy_exp, s_pos=s_pos)

  return df


### BM


In [14]:
# BM Function
def BM_2d(n_steps=1000, speed=6, s_pos=[0, 0]):
  """
  Arguments:
    n_steps: Desc1
    speed: Desc2
    s_pos: Desc3
  Returns:
    BM_2d_df: return
  """

  # Init velocity vector
  velocity = Vec2d(speed, 0)

  BM_2d_df = pd.DataFrame(columns=['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos':s_pos[0],
                          'y_pos':s_pos[1]}])

  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  for i in range(n_steps-1):
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
    velocity = velocity.rotated(turn_angle)

    temp_df = pd.DataFrame([{'x_pos': BM_2d_df.x_pos[i]+velocity.x,
                            'y_pos': BM_2d_df.y_pos[i]+velocity.y}])

    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  return BM_2d_df

### CRW

In [15]:
# CRW Function
def CRW_2d(n_steps=1000, speed=6, s_pos=[0, 0], CRW_exponent=0.9):
  """
  Arguments:
    n_steps: Desc1
    speed: Desc2
    s_pos: Desc3
  Returns:
    CRW_2d_df: DataFrme con
  """

  # Init velocity vector
  velocity = Vec2d(speed, 0)

  CRW_2d_df = pd.DataFrame(columns=['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos':s_pos[0],
                          'y_pos':s_pos[1]}])

  CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index=True)

  for i in range(n_steps-1):
    turn_angle = wrapcauchy.rvs(CRW_exponent)
    velocity = velocity.rotated(turn_angle)

    temp_df = pd.DataFrame([{'x_pos': CRW_2d_df.x_pos[i]+velocity.x,
                            'y_pos': CRW_2d_df.y_pos[i]+velocity.y}])

    CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index=True)

  return CRW_2d_df

### LF

In [16]:
# Levy Fly
def LF_2d(n_steps, speed, c, alpha, beta=1.0, loc=3.0, s_pos=[0, 0]):
  """
  Arguments:
    n_steps: Pasos
    speed:   Velocidad
    c:       Coeficiente Cauchy
    alpha:   levy alpha
    beta:    levy beta
    loc:     levy loc
  Returns:
    Levy_2d_df: DataFrme con
  """

  steps_cont = 0
  velocity = Vec2d(speed, 0)

  LS_2d = pd.DataFrame(columns=['x_pos',
                                'y_pos'])

  temp_df = pd.DataFrame([{'x_pos':s_pos[0],
                          'y_pos':s_pos[1]}])

  LS_2d = pd.concat([LS_2d, temp_df], ignore_index=True)

  while steps_cont < n_steps:

    # Elegir el angulo de giro
    turn_angle = wrapcauchy.rvs(c)
    velocity = velocity.rotated(turn_angle)

    # Numero de pasos
    step = int(levy_stable.rvs(alpha=alpha, beta=beta, loc=loc, scale=1, size=1))

    for _ in range(step):
      # Desplazamiento del caminador
      steps_cont += 1

      s_pos = [s_pos[0] + velocity.x,
               s_pos[1] + velocity.y]

      temp_df = pd.DataFrame([{'x_pos': s_pos[0],
                               'y_pos': s_pos[1]}])

      LS_2d = pd.concat([LS_2d, temp_df], ignore_index=True)

      if steps_cont >= n_steps:
        break

  return LS_2d

## Metrics

In [17]:
def PL_Metric(TDF):
  dis = np.array([distance.euclidean(TDF.iloc[i-1], TDF.iloc[i]) for i in range(1, TDF.shape[0])])
  pl= np.cumsum(dis)

  return pl


def MSD_Metric(TDF):
  MSD = np.empty(shape=(0))

  for n in range(1, TDF.shape[0]):
      ## start - Add your code here
      deltas = TDF.iloc[n:].values - TDF.iloc[:-n].values
      sq_distances = np.sum(deltas**2, axis=1)
      msd_value = np.mean(sq_distances)
      MSD = np.append(MSD, msd_value)
      ## end - Add your code here

  msd_df = pd.DataFrame({'MSD': MSD})

  return msd_df

## Plot


In [18]:
def plot_2d(fig_2d, x, y, name):
  fig_2d.add_trace(go.Scatter(
      x = x,
      y = y,
      marker = dict(size=2),
      line = dict(width=2),
      mode = 'lines',
      name = name,
      showlegend = True
  ))

  return fig_2d

def plot_3d(fig_3d, x, y, z, name):
  fig_3d.add_trace(go.Scatter3d(
      x = x,
      y = y,
      z = z,
      marker = dict(size=2),
      line = dict(width=2),
      mode = 'lines',
      name = name,
      showlegend = True
  ))

  return fig_3d

# Widgets

## Buttons

In [19]:
Trajectory_Selection = pnw.RadioButtonGroup(name='Buttons', value='BM', options=['BM', 'CRW', 'LF'])

n_steps = pnw.IntSlider(name='Number of steps', value=1000, step=1, start=1, end=10000)
speed = pnw.IntSlider(name='Speed', value=6, step=1, start=1, end=20)

cauchy_coef = pnw.FloatSlider(name='Cauchy Coef', value=0.7, step=0.01, start=0, end=2)
levy_exp = pnw.FloatSlider(name='Levy Exp', value=0.7, step=0.01, start=0, end=2)

x_pos = pnw.IntInput(name='X pos', width=100, value=0, step=1, start=0, end=100)
y_pos = pnw.IntInput(name='Y pos', width=100, value=0, step=1, start=0, end=100)

metrics = pnw.Select(name='Metrics', value='PL',options=['PL','MSD'])

## Front Functionality

In [20]:
def BM_View():
  return pn.Column(n_steps,
                   pn.Row(x_pos, y_pos),
                   speed,
                   metrics)

def CRW_View():
  return pn.Column(n_steps,
                   pn.Row(x_pos, y_pos),
                   speed,
                   cauchy_coef,
                   metrics)

def LF_View():
  return pn.Column(n_steps,
                   pn.Row(x_pos, y_pos),
                   speed,
                   cauchy_coef,
                   levy_exp,
                   metrics)


In [26]:
@pn.depends(Trajectory_Selection, n_steps, x_pos, y_pos, speed, cauchy_coef, levy_exp)
def Graph_3D_View(Trajectory_Selection, n_steps, x_pos, y_pos, speed, cauchy_coef, levy_exp):

  global df
  df = Trajectory_DF(Trajectory_Selection, n_steps, x_pos, y_pos, speed, cauchy_coef, levy_exp)

  fig = go.Figure()
  fig = plot_3d(fig, df.x_pos, df.y_pos, df.index, Trajectory_Selection + ' 3D Trajectory')

  return fig

@pn.depends(Trajectory_Selection, metrics)
def Metrics_View(Trajectory_Selection, metrics):
  fig = go.Figure()
  metric = []
  x = []
  global df

  if metrics == 'PL':
    metric = PL_Metric(df)
    x = np.arange(len(metric))+1

  elif metrics == 'MSD':
    metric = MSD_Metric(df)
    metric = metric['MSD']
    x = np.linspace(0, len(metric), 1000)

  fig = plot_2d(fig, x, metric, metrics)

  return fig

## Front

In [22]:
@pn.depends(Trajectory_Selection)
def Trajectory_Selection_Function(Trajectory_Selection):
  if (Trajectory_Selection == 'BM') :
    return BM_View
  elif (Trajectory_Selection == 'CRW'):
    return CRW_View
  elif (Trajectory_Selection == 'LF'):
    return LF_View

In [23]:

def View_3D():
  return pn.Column('3D Trajectory', Graph_3D_View)

def View_Metrics():
  return  pn.Column('Metrics', Metrics_View)

def View_Panel_Params():
  return pn.Column('Panel Params', Trajectory_Selection, Trajectory_Selection_Function)

def View_Graph():
  return pn.Row(View_3D, View_Metrics)

# Dashboard

In [ ]:
df = pd.DataFrame(columns=['x_pos', 'y_pos'])
pn.Column(pn.Row(View_Panel_Params, View_Graph))